1. Загрузка и предобработка данных

In [ ]:
import kagglehub
import pandas as pd
import numpy as np

# Загрузка данных
path = kagglehub.dataset_download("retailrocket/ecommerce-dataset")
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/retailrocket/ecommerce-dataset/versions/2


In [ ]:
# Чтение данных
interactions = pd.read_csv(f"{path}/events.csv")
items = pd.read_csv(f"{path}/item_properties_part1.csv")
category_tree = pd.read_csv(f"{path}/category_tree.csv")

# Предобработка данных
interactions.drop_duplicates(inplace=True)
items.drop_duplicates(inplace=True)
category_tree.drop_duplicates(inplace=True)

interactions.dropna(inplace=True)
items.dropna(inplace=True)
category_tree.dropna(inplace=True)

# Фильтруем покупки из взаимодействий
purchases = interactions[interactions['event'] == 'transaction'][['visitorid', 'itemid']]



In [ ]:
# Анализ данных
# Подсчет популярности товаров по количеству покупок
popular_items = purchases['itemid'].value_counts().reset_index()
popular_items.columns = ['itemid', 'purchase_count']

# Вывод топ-10 популярных товаров
print("Топ-10 популярных товаров по количеству покупок:")
print(popular_items.head(10))


Топ-10 популярных товаров по количеству покупок:
   itemid  purchase_count
0  461686             133
1  119736              97
2  213834              92
3  312728              46
4    7943              46
5  445351              45
6   48030              41
7  420960              38
8  248455              38
9   17478              37


2. Реализация простой рекомендательной системы

In [ ]:
# Реализация простой рекомендательной системы
def recommend_popular_items(user_id, top_n=10):
    """Функция рекомендует топ-N популярных товаров всем пользователям."""
    return popular_items.head(top_n)['itemid'].tolist()

# Пример использования
user_id_example = 12345
recommended_items = recommend_popular_items(user_id_example)
print(f"Рекомендованные товары для пользователя {user_id_example}: {recommended_items}")

Рекомендованные товары для пользователя 12345: [461686, 119736, 213834, 312728, 7943, 445351, 48030, 420960, 248455, 17478]


3. Оценка точности рекомендаций

In [ ]:

# Группировка покупок по пользователям
user_purchases = purchases.groupby('visitorid')['itemid'].apply(set).reset_index()
test_users = user_purchases.sample(n=100, random_state=42)

def evaluate_recommendations(test_users, recommend_func, top_n=10):
    accuracy_scores = []
    recall_scores = []

    for _, row in test_users.iterrows():
        actual_items = row['itemid']
        recommended_items = set(recommend_func(row['visitorid'], top_n))

        hits = len(actual_items & recommended_items)
        accuracy = hits / top_n if top_n > 0 else 0
        recall = hits / len(actual_items) if len(actual_items) > 0 else 0

        accuracy_scores.append(accuracy)
        recall_scores.append(recall)

    return np.mean(accuracy_scores), np.mean(recall_scores)

# Функция популярных рекомендаций
def recommend_popular_items(user_id, top_n=10):
    popular_items = purchases['itemid'].value_counts().index[:top_n]
    return popular_items.tolist()

accuracy, recall = evaluate_recommendations(test_users, recommend_popular_items, top_n=10)
print(f"Accuracy@10: {accuracy:.4f}")
print(f"Recall@10: {recall:.4f}")

# Сравнение с случайными рекомендациями
def recommend_random_items(user_id, top_n=10):
    return np.random.choice(purchases['itemid'].unique(), size=top_n, replace=False).tolist()

random_accuracy, random_recall = evaluate_recommendations(test_users, recommend_random_items, top_n=10)
print(f"Accuracy случайных рекомендаций: {random_accuracy:.100f}")
print(f"Recall случайных рекомендаций: {random_recall:.100f}")


Accuracy@10: 0.0060
Recall@10: 0.0354
Accuracy случайных рекомендаций: 0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Recall случайных рекомендаций: 0.0000000000


4. Кастомизированная рекомендательная система

In [ ]:
unique_buyers = purchases['visitorid'].unique()
print(unique_buyers)

[ 599528  121688  552148 ... 1155978 1050575  855941]


In [ ]:
# Получаем категории товаров из item_properties_part1
categories = items[items['property'] == 'category'][['itemid', 'value']]
categories.rename(columns={'value': 'category'}, inplace=True)

def recommend_personalized_items(user_id, top_n=10):
    user_history = purchases[purchases['visitorid'] == user_id]['itemid']

    if user_history.empty:
        print("Нет данных о покупках пользователя.")
        return []

    user_categories = categories[categories['itemid'].isin(user_history)]['category']

    category_items = categories[categories['category'].isin(user_categories)]['itemid']
    category_popular = purchases[purchases['itemid'].isin(category_items)]

    personalized_popular = category_popular['itemid'].value_counts().reset_index()
    personalized_popular.columns = ['itemid', 'purchase_count']

    return personalized_popular.head(top_n)['itemid'].tolist()


In [ ]:
# Пример вызова функции
user_id_example = 855941
print(recommend_personalized_items(user_id_example))


[]
